# Conformance Checking in Declare4Py

This tutorial explains how to perform the checking of a DECLARE model and how to browse the results.

In [1]:
import sys
import os
import pathlib

SCRIPT_DIR = pathlib.Path("..", "src").resolve()
sys.path.append(os.path.dirname(SCRIPT_DIR))

from src.declare4py.D4PyEventLog import D4PyEventLog
from src.declare4py.ProcessModels.DeclareModel import DeclareModel
from src.declare4py.ProcessModels.DeclareModel import DeclareModelTemplate

The DECLARE constraints supported by Declare4Py can be retrieved with the `get_unary_templates()` and `get_binary_templates` functions of the `DeclareModelTemplate` class:

In [2]:
unary_templates = DeclareModelTemplate.get_unary_templates()
binary_templates = DeclareModelTemplate.get_binary_templates()

print("Unary templates:")
print("-----------------")
for template in unary_templates:
    print(template.templ_str)
print("\n")

print("Binary templates:")
print("-----------------")
for template in binary_templates:
    print(template.templ_str)

Unary templates:
-----------------
Existence
Absence
Exactly
Init


Binary templates:
-----------------
Choice
Exclusive Choice
Responded Existence
Response
Alternate Response
Chain Response
Precedence
Alternate Precedence
Chain Precedence
Succession
Alternate Succession
Co-Existence
Chain Succession
Not Chain Succession
Not Co-Existence
Not Succession
Not Responded Existence
Not Response
Not Precedence
Not Chain Response
Not Chain Precedence


Notice that for the templates `Existence`, `Absence` and `Exactly` an additional parameter is necessary for the cardinality. This has to be encoded in the DECLARE `.decl` model with a numeric suffix, for example `Exactly2` or `Existence23`.

The next step is the parsing of the log and of the DECLARE model.

In [3]:
log_path = os.path.join("..", "tests", "Sepsis Cases.xes.gz")

event_log = D4PyEventLog()
event_log.parse_xes_log(log_path)

model_path = os.path.join("..", "tests", "resource", "files", "declare", "data_model.decl")
decl_model = DeclareModel().parse_from_file(model_path)

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

The DECLARE model can be inspected by getting all the activity names or the constraints. This information is returned as a list of strings.

In [4]:
model_activities = decl_model.get_model_activities()
model_constraints = decl_model.get_decl_model_constraints()

print("Model activities:")
print("-----------------")
for idx, act in enumerate(model_activities):
    print(idx, act)
print("\n")

print("Model constraints:")
print("-----------------")
for idx, constr in enumerate(model_constraints):
    print(idx, constr)

Model activities:
-----------------
0 ER Triage
1 ER Registration
2 ER Sepsis Triage
3 Leucocytes
4 CRP
5 LacticAcid
6 IV Antibiotics
7 Admission NC
8 IV Liquid
9 Release A
10 Return ER
11 Admission IC


Model constraints:
-----------------
0 Response[ER Registration, Leucocytes] | | |
1 Chain Response[ER Registration, ER Triage] |(A.DiagnosticArtAstrup is false) AND (A.SIRSCritHeartRate is true) AND (A.org:group is A) AND (A.DiagnosticBlood is true) AND (A.DisfuncOrg is false) AND (A.DiagnosticECG is true) AND (A.Age >= 45) AND (A.InfectionSuspected is true) AND (A.DiagnosticLacticAcid is true) AND (A.DiagnosticSputum is true) AND (A.Hypoxie is false) AND (A.DiagnosticUrinaryCulture is true) AND (A.DiagnosticLiquor is false) AND (A.SIRSCritTemperature is true) AND (A.Infusion is true) AND (A.Hypotensie is false) AND (A.DiagnosticUrinarySediment is true) AND (A.Oligurie is false) AND (A.Age <= 80) AND (A.SIRSCritTachypnea is true) AND (A.DiagnosticOther is false) AND (A.SIRSCritLeucos 

After importing the Declare4Py package and specified the paths of the log and of the DECLARE model, a `BasicMPDeclareConformanceChecking` object has to be instantiated.

In [5]:
from src.declare4py.ProcessMiningTasks.DeclareConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer


basic_checker = MPDeclareAnalyzer(consider_vacuity=False, log=event_log, declare_model=decl_model)
conf_check_res = basic_checker.run()

In [18]:
conf_check_res.get_metric(metric="num_activations")

conf_check_res.get_metric(metric="num_fulfillments")
ee=conf_check_res.get_metric(constr_id=13, metric="num_activations")
#decl_model.parsed_model.templatestype()
print(ee)

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Once the log and the DECLARE model are loaded, the `conformance_checking()` function will perform the model checking. The input boolean parameter `consider_vacuity=true` means that vacuously satisfied traces are considered as satisfied, violated otherwise. This function returns a Python dictionary containing the results indexed by trace in the log. Each key is a tuple containing the trace id and its name (that can be retrieved with `get_trace_keys()`). The value is a Python dictionary with keys the names of the constraints (that can be retrieved with `model_constraints()`) and values a `CheckerResult` object containing the number of pendings, activations, violations, fulfilments and the truth value of the trace for that constraint.

```
model_check_res = {trace_1: {constr_1: CheckerResult object, constr_2: CheckerResult object, ...},
                   trace_2: {constr_1: CheckerResult object, ... },
                    ...
                  }
```
The `CheckerResult` objects can be accessed by the attributes `num_pendings`, `num_activations`, `num_fulfillments`, `num_violations` and `state`.

In [ ]:
model_check_res = d4py.conformance_checking(consider_vacuity=False)

Let's inspect the results for the trace `(1, 'B')` and the constraint
`Chain Response[LacticAcid, Leucocytes] |A.LacticAcid <= 0.8 |T.Leucocytes >= 13.8 |0,2778,m`

In [ ]:
constr_id = 11
trace_id = (1, 'B')
print(f"Constraint to check: {model_constraints[constr_id]}")
print(f"Number of pendings: {model_check_res[trace_id][model_constraints[constr_id]].num_pendings}")
print(f"Number of activations: {model_check_res[trace_id][model_constraints[constr_id]].num_activations}")
print(f"Number of fulfilments: {model_check_res[trace_id][model_constraints[constr_id]].num_fulfillments}")
print(f"Number of violation: {model_check_res[trace_id][model_constraints[constr_id]].num_violations}")
print(f"Truth value: {model_check_res[trace_id][model_constraints[constr_id]].state}")

The checker results can be printed with the `print_conformance_results` function.